In [1]:
using ArchGDAL
using FreqTables
using LinearAlgebra
using DataFrames
using Dates
using Pipe
using Plots
using Rasters
using Serialization
using SparseArrays

include("util.jl")
include("geo_ana.jl")
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [2]:
path_spatial = "../dt_tmp/spatial_params_agg230.ser"

"../dt_tmp/spatial_params_agg230.ser"

## International airport location, index 

In [3]:
sp_pars = deserialize(path_spatial)
df = sp_pars.df
# Tambo International
lat = -26.12825796201514
lon = 28.242074092511
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# Cape Town 
lat =  -33.970502228847884
lon = 18.600228711334545
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# King Shaka 
lat =  -29.608764960536764
lon = 31.115368797913593
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")



argmin(dist): 11, dist: 9.631110668981826
argmin(dist): 7, dist: 5.508420257007044
argmin(dist): 62, dist: 8.80101225388816


# Transmission model 

In [4]:
n_sim = 5000

5000

In [5]:
R0 = 14.0
α = 0.05
#pc = 0.25
pc = 1.0

1.0

In [6]:
imp_ws_moz = CSV.read("../data/imp_ws_moz.csv", DataFrame, header=false)[:,1]
imp_ws_airport = CSV.read("../data/imp_ws_airport.csv", DataFrame, header=false)[:,1]
nothing

In [7]:
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [8]:
function post_processing(path_trans, path_spatial, pc)
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc)
    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    validate_meta_population_model(path_trans, path_spatial)
    remove_all_leaving_one(path_trans)
    return path_save
end

function run_all_patterns(;
        R0=1.0, α=0.05, pc=1.0, 
        imp_ws_moz=[1.0], imp_ws_airport=[1.0],
        n_sim=100,
        path_spatial=""
    )
    summary_info = []
    for pattern in ["population_size", "airport", "mozambique"]
        imp_ws = pattern=="airport" ? imp_ws_airport : imp_ws_moz
        path_trans = run_transmission_model(;
            R0=R0, α=α, pc=pc, imp_ws=imp_ws,
            n_sim=n_sim,
            path_spatial_params=path_spatial, 
            pattern=pattern
        )
        path_save = post_processing(path_trans, path_spatial, pc)
        rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
        push!(summary_info, rec)
    end
    println(summary_info)
end

run_all_patterns (generic function with 1 method)

In [9]:
run_all_patterns(
    R0=R0, α=α, pc=pc, imp_ws_moz=imp_ws_moz, imp_ws_airport=imp_ws_airport, 
    n_sim=n_sim,
    path_spatial=path_spatial,
    )

# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 1.0
  imp_ws: Array{Float64}((1517,)) [0.00811886524

Progress: 100%|█████████████████████████████████████████| Time: 1:39:21


5961.869641 seconds (57.23 M allocations: 16.024 TiB, 2.08% gc time, 0.01% compilation time)
Cum index:4
Actual ES coverage: 9.227133
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:18:17


../dt_tmp_res/20231121_232432.ser


...

7883.475204 seconds (84.04 M allocations: 26.112 TiB, 0.94% gc time)
Cum index:4
Actual ES coverage: 9.227133
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:16:19


../dt_tmp_res/20231122_030134.ser


Progress: 100%|█████████████████████████████████████████| Time: 0:02:55


Any["R0=14.0, α=0.05, pc=1.0, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231121_232432.ser", "R0=14.0, α=0.05, pc=1.0, n_sim=5000, pattern=airport, path: ../dt_tmp_res/20231122_003029.ser", "R0=14.0, α=0.05, pc=1.0, n_sim=5000, pattern=mozambique, path: ../dt_tmp_res/20231122_030134.ser"]


## Sensitivity analysis 

In [9]:
summary_info = []
pattern = "population_size"
for R0_sens in [10, 12, 16]
    path_trans = run_transmission_model(;
        R0=R0_sens, α=α, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0_sens), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

# of sites: 1517
SEIRMetaModelParams
  R0: Float64 10.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.6657789613848203
  pc: Float64 0.25
  imp_ws: Array{Float64}((1,)) [1.0]
Average va

Progress: 100%|█████████████████████████████████████████| Time: 1:29:01


3896.873467 seconds (37.18 M allocations: 8.799 TiB, 2.31% gc time, 0.01% compilation time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:23:25


../dt_tmp_res/20231117_101212.ser


...

9499.328785 seconds (72.45 M allocations: 21.943 TiB, 1.02% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:21:07


../dt_tmp_res/20231117_152940.ser


Progress: 100%|█████████████████████████████████████████| Time: 0:05:50


Any["R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231117_101212.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231117_122254.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231117_152940.ser"]


In [10]:
summary_info = []
for α_sens in [0.005, 0.010, 0.100, 0.500]
    path_trans = run_transmission_model(;
        R0=R0, α=α_sens, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern,
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0), α=$(α_sens), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.005
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Array{Float64}((1,)) [1.0]
Average v

Progress: 100%|█████████████████████████████████████████| Time: 1:31:16


5476.525449 seconds (48.54 M allocations: 13.345 TiB, 0.96% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:20:13


../dt_tmp_res/20231117_172755.ser


...

6939.712071 seconds (61.31 M allocations: 17.931 TiB, 12.85% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 1:37:26


../dt_tmp_res/20231118_013048.ser


Progress: 100%|█████████████████████████████████████████| Time: 0:04:07


Any["R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231117_172755.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231117_191759.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231117_215227.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231118_013048.ser"]


## Mozambique risk ordered ES. 

In [ ]:
imp_ws_moz_sorted = CSV.read("../data/imp_ws_moz_sorted.csv", DataFrame, header=false)[:,1]
imp_ws_airport_sorted = CSV.read("../data/imp_ws_airport_sorted.csv", DataFrame, header=false)[:,1]
path_spatial_sorted = "../dt_tmp/spatial_params_agg230_moz_sorted.ser"

In [ ]:
run_all_patterns(
    R0=R0, α=α, pc=pc, 
    imp_ws_moz=imp_ws_moz_sorted, 
    imp_ws_airport=imp_ws_airport_sorted,
    n_sim=n_sim,
    path_spatial=path_spatial_sorted,
    )

## Debug field

In [ ]:
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=imp_ws,
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern="population_size"
)
par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc)
#baseline_results(par_AFP, par_ES; 
#    path_trans=path_trans
#)

sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
validate_meta_population_model(path_trans, path_spatial)
remove_all_leaving_one(path_trans)

In [ ]:
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern="airport"
)
par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial)
sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
validate_meta_population_model(path_trans, path_spatial)
remove_all_leaving_one(path_trans)

In [ ]:
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=imp_ws,
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern="mozambique"
)
par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial)
sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
validate_meta_population_model(path_trans, path_spatial)
remove_all_leaving_one(path_trans)